# Plotting with Bokeh

-------


> This is a reworked version of tutorial by [Justin Bois](http://justinbois.github.io/bootcamp/2020/index.html). It contains modified training datasets, adopts content to Colab environment, and includes new sections. 

In [42]:
import pandas as pd
import numpy as np
import seaborn as sns

# Our main plotting package (must have explicit import of submodules)
import bokeh.io
import bokeh.plotting

# Enable viewing Bokeh plots in the notebook
bokeh.io.output_notebook()

<hr>

We have a couple nice data sets from the last few lessons and exercises, the data from the tongue strikes of frogs and facial matching data from people with sleep deprivation. We've learned how to use Pandas to handle the data sets and get what we want out of them. But I argue that what we want out of them most of the time is **plots**. Now is the time in the bootcamp where we learn how to take data set and plot them.

## The Python visualization landscape

Let us start by looking at some of the many plotting packages available in Python. In a [talk at PyCon in 2017](https://www.youtube.com/watch?v=FytuB8nFHPQ), [Jake VanderPlas](http://vanderplas.com), who is one of the authors of one of them ([Altair](https://altair-viz.github.io)), gave an overview of the Python visualization landscape. That landscape is depicted below, taken from [this visualization of it](https://github.com/rougier/python-visualization-landscape) by [Nicolas Rougier](http://www.labri.fr/perso/nrougier/). (It is from 2017, so it is a bit dated, and definitely not complete, notably missing [Panel](https://panel.holoviz.org), for example.)

<div style="margin: auto; width: 600px;">

![python_viz_landscape](http://justinbois.github.io/bootcamp/2020/_images/python_viz_landscape.png)

</div>

The landscape is divided into three main pods based on the low-level renderer of the graphics, JavaScript, [Matplotlib](https://matplotlib.org), and [OpenGL](https://www.opengl.org) (though Matplotlib is higher-level than JavaScript and OpenGL). We will not discuss packages based on OpenGL. Packages that use JavaScript for rendering are particularly well suited for interactivity in browsers. Interactivity and portability (accomplished by rendering in browsers) are key features of modern plotting libraries, so we will use JavaScript-based plotting in the bootcamp (as I do in my own work).

## High-level and low-level plotting packages

As a demonstration of what I mean by high-level and low-level plotting packages, let us first think about one of our tasks we did with Pandas using [Anscombe's quartet](https://en.wikipedia.org/wiki/Anscombe%27s_quartet) dataset by borrowing it from [Seaborn](https://seaborn.pydata.org/index.html) package:

In [3]:
df = sns.load_dataset("anscombe")

df.groupby('dataset')['x'].median()

dataset
I      9.0
II     9.0
III    9.0
IV     8.0
Name: x, dtype: float64

Literally three lines of code. Now what if we tried to do it without Pandas? I won't even go through the many lines of code necessary to read in the data. Consider just this one line.

```python
df.groupby('dataset')['x'].median()
```

There are elementary tasks that go into it if we were to code it up without using Pandas's delicious functionality. We can loop over the rows in the data frame with a `for` loop, check to see what the value of the `dataset` column is with an `if` statement, put the value in the `dataset` field into an appropriate array, and then, given those arrays, sort them and pull out the middle value. Under the hood, all of those steps take place, but because we use Pandas's **high-level functionality**, those details are invisible to us, and glad we are of that.

Now, say we want to make a plot of some data. You can imagine that there are many many steps to building that. One way you could build a plot is to hand-generate an [SVG file](https://en.wikipedia.org/wiki/Scalable_Vector_Graphics) that is a set of specifications for lines and circles and text and whatnot that comprises a plot. (I have actually done this before, writing a C program that hand-generated SVG, and it was paaaaainful.) That would be a very **low-level** way of generating a plot. Plotting libraries in Python usually take care of the rendering part for you, either rendering the plot as SVG, PDF, PNG, or other formats, including interactive ones that use [JavaScript](https://en.wikipedia.org/wiki/JavaScript) and [HTML Canvas](https://en.wikipedia.org/wiki/Canvas_element) that can be viewed in a browser. The plotting libraries then vary in their level of abstraction from the data set.

Lower-level plotting libraries typically are more customizable, but require more **[boilerplate code](https://en.wikipedia.org/wiki/Boilerplate_code)** to get your data plotted and are therefore more cumbersome. Higher-level plotting libraries aim to make it easier to move directly from a data frame to a plot. Because of this streamlining, though, they are often more difficult to customize.

The developers of [PyViz](http://pyviz.org/) made a nice graphic for this concept (Copyright PyViz authors, downloaded [here](http://pyviz.org/background.html)).

<div style="margin: auto; width: 500px;">

![shortcuts](http://justinbois.github.io/bootcamp/2020/_images/shortcuts.png)

</div>

Using a low-level plotting library, you can get to any graphic you like, but it takes many steps to do so. Using a high level library, you can rapidly get to many, if not most, of the graphics you like in very few steps. However, you cannot get to *all* graphics. In a layered approach, in which the higher level libraries give you access to the lower level customizations, you can get to any graphic, and can do so quickly. The layered approach requires proficiency in using the low-level and high-level libraries.

## High versus low

One debate was about whether to start with higher-level plotting or lower-level. We decided to do our plotting using [Bokeh](https://bokeh.pydata.org/), a low-level plotting library first for a few reasons.

1. Though low-level, generating plot you might like to construct is fairly straightforward. (Read: it's not that bad for quickly making plots.)
2. By being familiar with a lower-level plotting package, you can then take a layered approach as you learn a higher-level package.
3. We will discuss higher-level packages in short order, discussed in the next lesson.

Importantly, note that Bokeh's submodules often have to be explicitly imported, as we did in the code cell at the top of this notebook. Not also that if you want your plots to be viewable (and interactive) in the notebook, you need to execute

```python
bokeh.io.output_notebook()
```

at the top of the notebook (as we have done). 

## Bokeh's grammar and our first plot with Bokeh

Constructing a plot with Bokeh consists of four main steps.

1. Creating a figure on which to populate glyphs (symbols that represent data, e.g., dots for a scatter plot). Think of this figure as a "canvas" which sets the space on which you will "paint" your glyphs.
2. Defining a data source (almost always a tidy data frame) that is the reference used to place the glyphs.
3. Choose the kind of glyph you would like.
4. Annotate the columns of data source to determine how they are used to place (and possibly color, scale, etc.) the glyph.

After completing these steps, you need to render the graphic.

Let's go through these steps in generating a scatter plot.

1. Our first step is creating a figure, our "canvas." In creating the figure, we are implicitly thinking about what kind of representation for our data we want. That is, we have to specify axes and their labels. We might also want to specify the title of the figure, whether or not to have grid lines, and all sorts of other customizations. Naturally, we also want to specify the shape of the figure.

(Almost) all of this is accomplished in Bokeh by making a call to `bokeh.plotting.figure()` with the appropriate keyword arguments.

In [10]:
# Create the figure, stored in variable `p`
p = bokeh.plotting.figure(
    frame_width=400,
    frame_height=300,
    x_axis_label='X',
    y_axis_label='Y'
)

There are many more keyword attributes you can assign, including [all of those listed in the Bokeh Plot class](https://bokeh.pydata.org/en/latest/docs/reference/models/plots.html#bokeh.models.plots.Plot) and [the additional ones listed in the Bokeh Figure class](https://bokeh.pydata.org/en/latest/docs/reference/plotting.html#bokeh.plotting.figure.Figure).

2. Now that we have set up our canvas, we can decide on the data source. We will use the data frame, `df` as our data source. 

3. We will choose dots (or circles) as our glyph. This kind of glyph requires that we specify  which column of the data source will serve to place the glyphs along the $x$-axis and which will serve to place the glyphs along the $y$-axis.

4. We choose the `'x'` column to specify the $x$-coordinate of the glyph and the `'y'` column to specify the $y$-coordinate. We already made this decision when we set up our axis labels, but we did not necessarily have to make that decision at that point.

Steps 2-4 are accomplished by calling one of the **glyph methods** of the Bokeh `Figure` instance, `p`. Since we are choosing dots, the appropriate method is `p.circle()`, and we use the `source`, `x`, and `y` kwargs to specify the positions of the glyphs.

In [11]:
p.circle(
    source=df,
    x='x',
    y='y'
);

Now that we have built the plot, we can render it in the notebook using `bokeh.io.show()`.

In [12]:
bokeh.io.show(p)

In looking at the plot, notice a toolbar to right of the plot that enables you to zoom and pan within the plot.

## The importance of tidy data frames

It might be clear for you now that building a plot in this way requires that the data frame you use be tidy. The organization of tidy data is really what enables this and high level plotting functionality. There is a well-specified organization of the data.

## Code style in plot specifications

Specifications of plots often involves calls to functions with lots of keyword arguments to specify the plot, and this can get unwieldy without a clear style. You can develop your own style, maybe reading [Trey Hunner's blog post again](http://treyhunner.com/2017/07/craft-your-python-like-poetry/). I like to do the following.

1. Put the function call, like `p.circle(` or `p = bokeh.plotting.figure(` on the first line.
2. The closed parenthesis for the function call is on its own line, unindented.
3. Any arguments are given as kwargs (even if they can also be specified as positional arguments) at one level of indentation.

Note that you *cannot* use method chaining when instantiating figures or populating glyphs.

If you adhere to a style (which is roughly the style imposed by [Black](https://black.readthedocs.io/en/stable/)), it makes your code cleaner and easier to read.

## Coloring with other dimensions

Let's say we wanted to make the same plot, but color circles for datasets. To do this, we take advantage of two features of Bokeh.

1. We can make multiple calls to `p.circle()` to populate more and more glyphs.
2. `p.circle()`, like all of the glyph methods, has many keyword arguments, including `color` and `legend_label`, which will enable us to color the glyphs and include a legend.

We can loop through the data frame grouped by `'dataset'` and populate the glyphs as we go along.

In [16]:
# For convenience
x = 'x'
y = 'y'

# Make figure
p = bokeh.plotting.figure(
    frame_width=400,
    frame_height=300,
    x_axis_label=x,
    y_axis_label=y,
)

# Add glyphs
p.circle(
    source=df.loc[df['dataset']=='I', :],
    x=x,
    y=y,
    legend_label='I',
    color='red'
)

p.circle(
    source=df.loc[df['dataset']=='II', :],
    x=x,
    y=y,
    legend_label='II',
    color='blue'

)

p.circle(
    source=df.loc[df['dataset']=='III', :],
    x=x,
    y=y,
    legend_label='III',    
    color='green'

)

p.circle(
    source=df.loc[df['dataset']=='IV', :],
    x=x,
    y=y,
    legend_label='IV',
    color='purple'

)


bokeh.io.show(p)

We can automate addiung glyphs using a loop. This way we do not have to explicitly set `p.circle` for each kind of `dataset`:

In [19]:
# For convenience
x = 'x'
y = 'y'

# Make figure
p = bokeh.plotting.figure(
    frame_width=400,
    frame_height=300,
    x_axis_label=x,
    y_axis_label=y,
)

# Add glyphs

for dataset in df['dataset'].unique():

    p.circle(
        source=df.loc[df['dataset']==dataset, :],
        x=x,
        y=y,
        legend_label=dataset,
        color='red'
    )

bokeh.io.show(p)

Of course the problem that they are all read and we need to differentiate. [Bokeh palettes](https://docs.bokeh.org/en/latest/docs/reference/palettes.html) will help us with that:

In [25]:
from bokeh.palettes import Category10
colors = Category10

# For convenience
x = 'x'
y = 'y'

# Make figure
p = bokeh.plotting.figure(
    frame_width=400,
    frame_height=300,
    x_axis_label=x,
    y_axis_label=y,
)

# Add glyphs

for i, dataset in enumerate(df['dataset'].unique()):

    p.circle(
        source=df.loc[df['dataset']==dataset, :],
        x=x,
        y=y,
        legend_label=dataset,
        color=colors[4][i]
    )

bokeh.io.show(p)

We got the plot we wanted, but the legend is clashing with the data. Fortunately, Bokeh allows us to set attributes of the figure whenever we like. We can therefore set the legend position to be in the upper left corner. We will also set the `click_policy` for the legend to be `'hide'`, which will hide glyphs if you click the legend, which can be convenient for viewing cluttered plots (though this one is not cluttered, really).

In [26]:
p.legend.location = 'top_left'
p.legend.click_policy = 'hide'

bokeh.io.show(p)

## Adding tooltips

Bokeh's interactivity is one of its greatest strengths. We might also like to have access to other information in our (tidy) data source if we hover over a glyph. Let's say we want to know the dataset and coordinates. We can tell Bokeh to give us this information by adding **tooltips** when we instantiate the figure.

The syntax for a tooltip is a list of 2-tuples, where each tuple represents the tooltip you want. The first entry in the tuple is the label and the second is the column from the data source that has the values. The second entry must be preceded with an `@` symbol signifying that it is a field in the data source and not field that is intrinsic to the plot, which is preceded with a `$` sign. If there are spaces in the column heading, enclose the column name in braces. (See the [documentation for tooltip specification](https://bokeh.pydata.org/en/latest/docs/user_guide/tools.html#basic-tooltips) for more information.)

In [28]:
# For convenience
x = 'x'
y = 'y'

# Make figure
p = bokeh.plotting.figure(
    frame_width=400,
    frame_height=300,
    x_axis_label=x,
    y_axis_label=y,
    tooltips=[
        ('dataset', '@dataset'),
        ('x', '@x'), 
        ('y', '@y'), 
    ]
)

# Add glyphs

for i, dataset in enumerate(df['dataset'].unique()):

    p.circle(
        source=df.loc[df['dataset']==dataset, :],
        x=x,
        y=y,
        legend_label=dataset,
        color=colors[4][i]
    )

p.legend.location = 'top_left'
p.legend.click_policy = 'hide'

bokeh.io.show(p)

## Creating multiple plots

Sometimes you want to separate visualization into multiple plots. In the case of Anscombe quartet we might want to plot each dataset separately. This can be done by defining multiple `figure()` instances and arranging them in rows:

In [34]:
from bokeh.layouts import row

# For convenience
x = 'x'
y = 'y'

figs = []


for i, dataset in enumerate(df['dataset'].unique()):

    # Make figure
    
    p = bokeh.plotting.figure(
        frame_width=400,
        frame_height=300,
        x_axis_label=x,
        y_axis_label=y,
        tooltips=[
            ('dataset', '@dataset'),
            ('x', '@x'), 
            ('y', '@y'), 
        ]
    )
    
    # Add glyphs

    p.circle(
        source=df.loc[df['dataset']==dataset, :],
        x=x,
        y=y,
        legend_label=dataset,
        color=colors[4][i]
    )
    figs.append(p)

bokeh.io.show(row(figs))

columns:

In [35]:
from bokeh.layouts import column

# For convenience
x = 'x'
y = 'y'

figs = []


for i, dataset in enumerate(df['dataset'].unique()):

    # Make figure
    
    p = bokeh.plotting.figure(
        frame_width=400,
        frame_height=300,
        x_axis_label=x,
        y_axis_label=y,
        tooltips=[
            ('dataset', '@dataset'),
            ('x', '@x'), 
            ('y', '@y'), 
        ]
    )
    
    # Add glyphs

    p.circle(
        source=df.loc[df['dataset']==dataset, :],
        x=x,
        y=y,
        legend_label=dataset,
        color=colors[4][i]
    )
    figs.append(p)

bokeh.io.show(column(figs))

or as a grid:

In [51]:
from bokeh.layouts import gridplot

# For convenience
x = 'x'
y = 'y'

figs = []


for i, dataset in enumerate(df['dataset'].unique()):

    # Make figure
    
    p = bokeh.plotting.figure(
        frame_width=400,
        frame_height=300,
        x_axis_label=x,
        y_axis_label=y,
        tooltips=[
            ('dataset', '@dataset'),
            ('x', '@x'), 
            ('y', '@y'), 
        ]
    )
    
    # Add glyphs

    p.circle(
        source=df.loc[df['dataset']==dataset, :],
        x=x,
        y=y,
        legend_label=dataset,
        color=colors[4][i]
    )
    figs.append(p)
grid = gridplot( [[figs[0],figs[1]],[figs[2],figs[3]]] )
bokeh.io.show(grid)

## Linking multiple plots

Bokeh can link multiple plots together so that intercativ eexploration of one plot keeps th eentire pucture in sync:

In [54]:
from bokeh.layouts import gridplot

# For convenience
x = 'x'
y = 'y'

figs = []


for i, dataset in enumerate(df['dataset'].unique()):

    # Make figure
    if i == 0:
        p = bokeh.plotting.figure(
            frame_width=400,
            frame_height=300,
            x_axis_label=x,
            y_axis_label=y,
            tooltips=[
                ('dataset', '@dataset'),
                ('x', '@x'), 
                ('y', '@y'), 
            ],
        )
    else:
        p = bokeh.plotting.figure(
            frame_width=400,
            frame_height=300,
            x_axis_label=x,
            y_axis_label=y,
            tooltips=[
                ('dataset', '@dataset'),
                ('x', '@x'), 
                ('y', '@y'), 
            ],
            x_range = figs[0].x_range, 
            y_range = figs[0].y_range
        )

    
    # Add glyphs

    p.circle(
        source=df.loc[df['dataset']==dataset, :],
        x=x,
        y=y,
        legend_label=dataset,
        color=colors[4][i]
    )
    figs.append(p)
grid = gridplot( [[figs[0],figs[1]],[figs[2],figs[3]]] )
bokeh.io.show(grid)

A complete example with regression lines:

In [65]:
from bokeh.plotting import figure,show
figs = []
plot_options = dict(plot_height = 400, plot_width = 600)


for i, dataset in enumerate(df['dataset'].unique()):
    #title = 'Dataset = {}'.format(dataset)
    if i == 0:
        fig = figure(**plot_options, title='Dataset = {}'.format(dataset))
    else:
        fig = figure(**plot_options, x_range = figs[0].x_range, y_range = figs[0].y_range,title='Dataset = {}'.format(dataset))
    x = df[df['dataset']==dataset]['x']
    y = df[df['dataset']==dataset]['y']
    slope,intercept = np.polyfit(x,y, 1)
    y_fit = [slope * i + intercept for i in x]
    source = ColumnDataSource({'x':x, 'y':y,'y_fit':y_fit})
    fig.circle(x='x',y='y',source=source, color = colors[4][i],size=10)
    fig.line(x='x',y='y_fit',source=source,color = colors[4][i],line_width=4,legend_label='y='+str(round(slope,2))+'x+'+str(round(intercept,2)))

    figs.append(fig)
grid = gridplot( [[figs[0],figs[1]],[figs[2],figs[3]]] )
show(grid)


## Computing environment

In [33]:
!pip install watermark
%load_ext watermark
%watermark -v -p pandas,bokeh

The watermark extension is already loaded. To reload it, use:
  %reload_ext watermark
CPython 3.6.9
IPython 5.5.0

pandas 1.1.5
bokeh 2.1.1
